#Trabajo Práctico 1 : Reservas de Hotel

## Preprocesamientio de datos

In [95]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st

import warnings
warnings.simplefilter(action= 'ignore', category=FutureWarning)

### Dataset Train

In [96]:
ds_hotel_train = pd.read_csv('hotels_train.csv')
df_hotel_train = ds_hotel_train.copy()

ds_hotel_test = pd.read_csv('hotels_test.csv')
df_hotel_test = ds_hotel_test.copy()

In [97]:
print("El dataframe tiene "f" {df_hotel_train.shape[0]} ""filas y "f"{df_hotel_train.shape[1]} ""columnas.")

El dataframe tiene  61913 filas y 31 columnas.


In [98]:
df_hotel_train[df_hotel_train.duplicated(keep = False)]

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,id,is_canceled


In [99]:
df_hotel_train.isna().sum()

hotel                                 0
lead_time                             0
arrival_date_year                     0
arrival_date_month                    0
arrival_date_week_number              0
arrival_date_day_of_month             0
stays_in_weekend_nights               0
stays_in_week_nights                  0
adults                                0
children                              4
babies                                0
meal                                  0
country                             221
market_segment                        0
distribution_channel                  0
is_repeated_guest                     0
previous_cancellations                0
previous_bookings_not_canceled        0
reserved_room_type                    0
assigned_room_type                    0
booking_changes                       0
deposit_type                          0
agent                              7890
company                           58761
days_in_waiting_list                  0


In [100]:
#COUNTRY
df_hotel_train.loc[df_hotel_train['country'].isnull(), 'country'] = 'PRT'

#AGENT
df_hotel_train.loc[df_hotel_train['agent'].isnull(), 'agent'] = 0
df_hotel_train['agent'] = df_hotel_train['agent'].astype(int)

#COMPANY
df_hotel_train.drop('company', axis = 'columns', inplace = True)
df_hotel_train.reset_index(drop = True)

#ADULTS
df_hotel_train.drop(df_hotel_train[df_hotel_train['adults'] == 0].index, axis = 'index', inplace = True)
df_hotel_train.drop(df_hotel_train[df_hotel_train['adults'] > 3].index, axis = 'index', inplace = True)
df_hotel_train.reset_index(drop = True)

#ADR
df_hotel_train.drop(df_hotel_train[df_hotel_train['adr'] <= 0].index, axis = 'index', inplace = True)
df_hotel_train.reset_index(drop = True)

media_adr = np.mean(df_hotel_train.adr)
std_adr = np.std(df_hotel_train.adr)
df_hotel_train['z_adr'] = (df_hotel_train.adr - media_adr)/std_adr
df_hotel_train['z_adr'] = st.zscore(df_hotel_train.adr)
df_hotel_train.drop(df_hotel_train[df_hotel_train['adr'] >= 400].index, axis = 'index', inplace = True)
df_hotel_train.drop(df_hotel_train[df_hotel_train['z_adr'] > 3].index, axis = 'index', inplace = True)
df_hotel_train.drop(df_hotel_train[df_hotel_train['z_adr'] < -2].index, axis = 'index', inplace = True)
df_hotel_train.drop(labels='z_adr', inplace = True, axis = 1)
df_hotel_train.reset_index(drop = True)

#BABIES
df_hotel_train.drop(df_hotel_train[df_hotel_train['babies'] >= 9].index, axis = 'index', inplace = True)
df_hotel_train.reset_index(drop = True)

#CHILDREN
df_hotel_train.loc[df_hotel_train['children'].isnull(), 'children'] = 0
df_hotel_train.loc[df_hotel_train['children'] < 0, 'children'] = 0
df_hotel_train.drop(df_hotel_train[df_hotel_train['children'] >= 10].index, axis = 'index', inplace = True)
df_hotel_train.reset_index(drop = True)

#PREVIOUS_CANCELLATIONS
df_hotel_train.drop(df_hotel_train[df_hotel_train['previous_cancellations'] >= 2].index, axis = 'index', inplace = True)
df_hotel_train.reset_index(drop = True)

#PREVIOUS_BOOKINGS_NOT_CANCELLED
df_hotel_train.drop('previous_bookings_not_canceled', axis = 'columns', inplace = True)
df_hotel_train.reset_index(drop = True)

#REQUIRED_CAR_PARKING_SPACES
df_hotel_train.loc[(df_hotel_train['adults'] == 2) & (df_hotel_train['required_car_parking_spaces'] == 3), 'required_car_parking_spaces'] = 2
df_hotel_train.loc[(df_hotel_train['adults'] == 2) & (df_hotel_train['required_car_parking_spaces'] == 8), 'required_car_parking_spaces'] = 2

#TOTAL_OF_SPECIAL_REQUESTS
df_hotel_train.loc[(df_hotel_train['total_of_special_requests'] >= 4), 'total_of_special_requests'] = 1

In [101]:
df_hotel_train['total_days'] = df_hotel_train['stays_in_week_nights'] + df_hotel_train['stays_in_weekend_nights']
df_hotel_train.drop(df_hotel_train[(df_hotel_train['assigned_room_type'] == 'K') & (df_hotel_train['adr'] > 160)].index, axis = 'index', inplace = True)
df_hotel_train.drop(df_hotel_train[(df_hotel_train['assigned_room_type'] == 'I') & (df_hotel_train['adr'] > 210)].index, axis = 'index', inplace = True)
df_hotel_train.drop(df_hotel_train[(df_hotel_train['assigned_room_type'] == 'B') & (df_hotel_train['adr'] > 160)].index, axis = 'index', inplace = True)
df_hotel_train.drop(df_hotel_train[(df_hotel_train['assigned_room_type'] == 'B') & (df_hotel_train['adr'] < 30)].index, axis = 'index', inplace = True)
df_hotel_train.drop('assigned_room_type', axis = 'columns', inplace = True)
df_hotel_train.reset_index(drop = True)

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,id,is_canceled,total_days
0,City Hotel,49,2016,September,37,5,1,2,1,0.0,...,No Deposit,9,0,Transient,115.50,0,1,7aa4cc6b-b92c-4061-b21d-4f9e0cac4689,1,3
1,Resort Hotel,4,2015,October,44,31,0,1,2,0.0,...,No Deposit,0,0,Transient,42.00,1,0,b1428f80-c56c-4ae4-91a9-6962edae08b1,0,1
2,City Hotel,25,2016,October,42,13,0,1,2,2.0,...,No Deposit,9,0,Transient,229.00,0,2,2226c018-ea20-4c4e-8f71-ea66aee70d7c,0,1
3,City Hotel,26,2016,December,51,12,2,5,2,0.0,...,Non Refund,154,0,Transient,75.00,0,0,314f56dc-2e4d-4298-9bdd-27bb008bffff,1,7
4,City Hotel,104,2017,March,12,19,2,5,2,0.0,...,No Deposit,20,0,Transient-Party,75.00,0,1,28c75445-c603-413a-babf-ad377018b592,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59635,Resort Hotel,24,2017,January,4,27,0,2,2,0.0,...,No Deposit,240,0,Transient,48.00,0,1,32bffef5-5404-4e86-9f39-fd2282e1ad7f,1,2
59636,City Hotel,0,2016,September,36,2,0,1,1,0.0,...,No Deposit,0,0,Transient,149.00,0,0,cc922a1c-356e-42a5-a984-0b1af03c2f5c,0,1
59637,City Hotel,10,2017,May,19,8,1,2,1,0.0,...,No Deposit,0,0,Transient,95.00,0,0,76fb0e83-59df-43ee-9c33-aec51030ed45,0,3
59638,Resort Hotel,143,2015,October,41,10,2,5,2,0.0,...,Non Refund,96,0,Transient,36.05,0,0,01906859-1094-4e37-b13e-b015b594d159,1,7


In [102]:
#ARRIVAL_DATE_MONTH
meses_a_numeros = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7,
    'August': 8,
    'September': 9,
    'October': 10,
    'November': 11,
    'December': 12
}
df_hotel_train.arrival_date_month = df_hotel_train.arrival_date_month.map(meses_a_numeros)

In [103]:
#ID
df_hotel_train.drop('id', axis = 'columns', inplace = True)

In [104]:
df_hotel_train.isna().sum()

hotel                          0
lead_time                      0
arrival_date_year              0
arrival_date_month             0
arrival_date_week_number       0
arrival_date_day_of_month      0
stays_in_weekend_nights        0
stays_in_week_nights           0
adults                         0
children                       0
babies                         0
meal                           0
country                        0
market_segment                 0
distribution_channel           0
is_repeated_guest              0
previous_cancellations         0
reserved_room_type             0
booking_changes                0
deposit_type                   0
agent                          0
days_in_waiting_list           0
customer_type                  0
adr                            0
required_car_parking_spaces    0
total_of_special_requests      0
is_canceled                    0
total_days                     0
dtype: int64

In [105]:
print("El dataframe tiene "f" {df_hotel_train.shape[0]} ""filas y "f"{df_hotel_train.shape[1]} ""columnas.")

El dataframe tiene  59640 filas y 28 columnas.


### Dataset Test

In [106]:
print("El dataframe tiene "f" {df_hotel_test.shape[0]} ""filas y "f"{df_hotel_test.shape[1]} ""columnas.")

El dataframe tiene  26535 filas y 31 columnas.


In [107]:
df_hotel_test.isna().sum()

hotel                                 0
lead_time                             0
arrival_date_year                     0
arrival_date_month                    0
arrival_date_week_number              0
arrival_date_day_of_month             0
stays_in_weekend_nights               0
stays_in_week_nights                  0
adults                                0
children                              0
babies                                0
meal                                  0
country                              95
market_segment                        0
distribution_channel                  0
is_repeated_guest                     0
previous_cancellations                0
previous_bookings_not_canceled        0
reserved_room_type                    0
assigned_room_type                    0
booking_changes                       0
deposit_type                          0
agent                              3363
company                           25218
days_in_waiting_list                  0


In [108]:
#AGENT
df_hotel_test.fillna({'agent':0}, inplace = True)

#ASSIGNED_ROOM_TYPE
df_hotel_test.drop('assigned_room_type', axis = 'columns', inplace = True)

In [109]:
#ARRIVAL_DATE_MONTH
meses_a_numeros = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7,
    'August': 8,
    'September': 9,
    'October': 10,
    'November': 11,
    'December': 12
}
df_hotel_test.arrival_date_month = df_hotel_test.arrival_date_month.map(meses_a_numeros)

In [110]:
#RESERVED_ROOM_TYPE
habitacion_train = df_hotel_train.reserved_room_type.unique().tolist()
habitacion_test = df_hotel_test.reserved_room_type.unique().tolist()
habitacion_train_test = set(habitacion_train).difference(set(habitacion_test))
habitacion_test_train = set(habitacion_test).difference(set(habitacion_train))
dif_habitacion_test_train = {x:'A' for x in habitacion_test_train}
dif_habitacion_train_test = {x:'A' for x in habitacion_train_test}
df_hotel_train['reserved_room_type'].replace(dif_habitacion_train_test, inplace = True)
df_hotel_test['reserved_room_type'].replace(dif_habitacion_test_train, inplace = True)

In [111]:
dormitorio_letra_a_numero = {
    'A': 1,
    'B': 2,
    'C': 3,
    'D': 4,
    'E': 5,
    'F': 6,
    'G': 7,
    'H': 8,
    'I': 9,
    'K': 10,
    'L': 11,
    'P': 12
}
df_hotel_test.reserved_room_type = df_hotel_test.reserved_room_type.map(dormitorio_letra_a_numero)

In [112]:
dormitorio_letra_a_numero = {
    'A': 1,
    'B': 2,
    'C': 3,
    'D': 4,
    'E': 5,
    'F': 6,
    'G': 7,
    'H': 8,
    'I': 9,
    'K': 10,
    'L': 11,
    'P': 12
}
df_hotel_train.reserved_room_type = df_hotel_train.reserved_room_type.map(dormitorio_letra_a_numero)

In [113]:
#DISTRIBUTION_CHANNEL

distribution_channel_train = df_hotel_train.distribution_channel.unique().tolist()
distribution_channel_test = df_hotel_test.distribution_channel.unique().tolist()
distribution_channel_train_test = set(distribution_channel_train).difference(set(distribution_channel_test))
distribution_channel_test_train = set(distribution_channel_test).difference(set(distribution_channel_train))
dif_distribution_channel_test_train = {x:'TA/TO' for x in distribution_channel_test_train}
dif_distribution_channel_train_test = {x:'TA/TO' for x in distribution_channel_train_test}
df_hotel_train['distribution_channel'].replace(dif_distribution_channel_train_test, inplace = True)
df_hotel_test['distribution_channel'].replace(dif_distribution_channel_test_train, inplace = True)

In [114]:
#COUNTRY
country_top_train = df_hotel_train['country'].value_counts()
top_20_country_train = country_top_train.head(20).index.tolist()
df_hotel_train['country'] = df_hotel_train['country'].apply(lambda x: x if x in top_20_country_train else 'other')
df_hotel_test['country'] = df_hotel_test['country'].apply(lambda x: x if x in top_20_country_train else 'other')

In [115]:
#MARKET_SEGMENT

market_segment_train = df_hotel_train.market_segment.unique().tolist()
market_segment_test = df_hotel_test.market_segment.unique().tolist()
market_segment_train_test = set(market_segment_train).difference(set(market_segment_test))
market_segment_test_train = set(market_segment_test).difference(set(market_segment_train))
dif_market_segment_test_train = {x:'Online TA' for x in market_segment_test_train}
dif_market_segment_train_test = {x:'Online TA' for x in market_segment_train_test}
df_hotel_train['market_segment'].replace(dif_market_segment_train_test, inplace = True)
df_hotel_test['market_segment'].replace(dif_market_segment_test_train, inplace = True)

In [116]:
#ID
df_hotel_test.drop('id', axis = 'columns', inplace = True)

In [117]:
#COMPANY
df_hotel_test.drop('company', axis = 'columns', inplace = True)

#AGENT
#AGENT
df_hotel_test.loc[df_hotel_test['agent'].isnull(), 'agent'] = 0
df_hotel_test['agent'] = df_hotel_test['agent'].astype(int)

#COUNTRY
df_hotel_test.loc[df_hotel_test['country'].isnull(), 'country'] = 'PRT'

#CHILDREN
df_hotel_test.loc[df_hotel_test['children'] < 0, 'children'] = 0
df_hotel_test['children'] = df_hotel_test['children'].astype(int)

#PREVIOUS_BOOKINGS_NOT_CANCELED
df_hotel_test.drop('previous_bookings_not_canceled', axis = 'columns', inplace = True)

#STAYS_IN_WEEK_NIGHTS & STAYS_IN_WEEKEND_NIGHTS
df_hotel_test['total_days'] = df_hotel_test['stays_in_week_nights'] + df_hotel_test['stays_in_weekend_nights']

#RESERVATION_STATUS_DATE
df_hotel_test.drop('reservation_status_date', axis = 'columns', inplace = True)


In [118]:
columns_modificar_train = df_hotel_train.dtypes[(df_hotel_train.dtypes != 'int64') & (df_hotel_train.dtypes != 'float64')].index
print(columns_modificar_train.tolist())

['hotel', 'meal', 'country', 'market_segment', 'distribution_channel', 'deposit_type', 'customer_type']


In [119]:
columns_modificar_test = df_hotel_test.dtypes[(df_hotel_test.dtypes != 'int64') & (df_hotel_test.dtypes != 'float64')].index
print(columns_modificar_test.tolist())

['hotel', 'meal', 'country', 'market_segment', 'distribution_channel', 'deposit_type', 'customer_type']


### One hot Enconding

In [120]:
df_hotel_train = pd.get_dummies(df_hotel_train, columns = columns_modificar_train, drop_first = True)
df_hotel_test = pd.get_dummies(df_hotel_test, columns = columns_modificar_test, drop_first = True)

In [122]:
faltan_columnas = list(sorted(set(df_hotel_test.columns) - set(df_hotel_train.columns)))
agregar_columnas = list(sorted(set(df_hotel_train.columns) - set(df_hotel_test.columns)))
print(faltan_columnas)
print(agregar_columnas)

[]
['is_canceled']


In [123]:
df_hotel_test.isna().sum()

lead_time                        0
arrival_date_year                0
arrival_date_month               0
arrival_date_week_number         0
arrival_date_day_of_month        0
stays_in_weekend_nights          0
stays_in_week_nights             0
adults                           0
children                         0
babies                           0
is_repeated_guest                0
previous_cancellations           0
reserved_room_type               0
booking_changes                  0
agent                            0
days_in_waiting_list             0
adr                              0
required_car_parking_spaces      0
total_of_special_requests        0
total_days                       0
hotel_Resort Hotel               0
meal_FB                          0
meal_HB                          0
meal_SC                          0
meal_Undefined                   0
country_BEL                      0
country_BRA                      0
country_CHE                      0
country_CHN         

In [124]:
df_hotel_train.isna().sum()

lead_time                        0
arrival_date_year                0
arrival_date_month               0
arrival_date_week_number         0
arrival_date_day_of_month        0
stays_in_weekend_nights          0
stays_in_week_nights             0
adults                           0
children                         0
babies                           0
is_repeated_guest                0
previous_cancellations           0
reserved_room_type               0
booking_changes                  0
agent                            0
days_in_waiting_list             0
adr                              0
required_car_parking_spaces      0
total_of_special_requests        0
is_canceled                      0
total_days                       0
hotel_Resort Hotel               0
meal_FB                          0
meal_HB                          0
meal_SC                          0
meal_Undefined                   0
country_BEL                      0
country_BRA                      0
country_CHE         

In [125]:
df_hotel_train.to_csv('df_hotel_train_clean.csv', index = False)

In [126]:
df_hotel_test.to_csv('df_hotel_test_clean.csv', index = False)